# Dashboard Interativo - Qualidade do Ar e Saúde
Este notebook gera visualizações interativas com base em dados simulados sobre poluentes e saúde pública.

In [ ]:
import pandas as pd

import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, udf
from pyspark.sql.types import StringType
import plotly.express as px

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


from pyspark.sql import SparkSession
# etc...
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, udf
from pyspark.sql.types import StringType

spark = SparkSession.builder \
    .appName("QualidadeDoAr_1000Registros") \
    .master("local[*]") \
    .getOrCreate()

print("SparkSession criada com sucesso!")

SparkSession criada com sucesso!


In [ ]:
spark_df = spark.read.csv(
    'C:\\Users\\Ericp\\.vscode\\dashboard_qualidade_ar.csv',
header=True,
inferSchema=True
)

print("Schema do DataFrame Spark:")
spark_df.printSchema()

Schema do DataFrame Spark:
root
 |-- id: integer (nullable = true)
 |-- data: date (nullable = true)
 |-- cidade: string (nullable = true)
 |-- pm2_5: double (nullable = true)
 |-- pm10: double (nullable = true)
 |-- no2: double (nullable = true)
 |-- so2: double (nullable = true)
 |-- o3: double (nullable = true)
 |-- co: double (nullable = true)
 |-- nh3: double (nullable = true)
 |-- pb: double (nullable = true)
 |-- umidade: double (nullable = true)
 |-- temperatura: double (nullable = true)
 |-- casos_problemas_respiratorios: integer (nullable = true)



In [ ]:
poluentes_cols = ['pm2_5', 'pm10', 'no2', 'so2', 'o3', 'co', 'nh3', 'pb']

spark_df_processed = spark_df.withColumn(
    'MediaPoluentes',
    sum(col(c) for c in poluentes_cols) / len(poluentes_cols)
)

In [ ]:
def classificar_qualidade_ar(media):
    if media < 25:
        return 'Boa'
    elif media < 40:
        return 'Moderada'
    elif media < 55:
        return 'Ruim'
    else:
        return 'Muito Ruim'

classificar_udf = udf(classificar_qualidade_ar, StringType())

spark_df_final = spark_df_processed.withColumn(
    'QualidadeAr',
    classificar_udf(col('MediaPoluentes'))
)

print("Processamento com Spark finalizado.")
spark_df_final.select('data', 'cidade', 'MediaPoluentes', 'QualidadeAr').show(5)

Processamento com Spark finalizado.
+----------+--------------+------------------+-----------+
|      data|        cidade|    MediaPoluentes|QualidadeAr|
+----------+--------------+------------------+-----------+
|2022-01-01|Belo Horizonte|30.024250000000002|   Moderada|
|2022-01-02|Belo Horizonte|29.791000000000004|   Moderada|
|2022-01-03|  Porto Alegre|          26.85825|   Moderada|
|2022-01-04|Belo Horizonte|          33.40775|   Moderada|
|2022-01-05|  Porto Alegre|         33.066375|   Moderada|
+----------+--------------+------------------+-----------+
only showing top 5 rows


In [ ]:
df_plotly = spark_df_final.toPandas()

df_plotly['data'] = pd.to_datetime(df_plotly['data'])

print(f"DataFrame convertido para Pandas com {len(df_plotly)} registros.")

DataFrame convertido para Pandas com 1000 registros.


In [22]:
# ==============================================================================
# ESTRATÉGIA FINAL: RECRIAÇÃO COMPLETA DO GRÁFICO A CADA CLIQUE
# ==============================================================================

# Importações necessárias, incluindo Output e clear_output
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import VBox, HBox
from IPython.display import display, clear_output

# 1. Preparar o DataFrame e Widgets (sem mudanças)
df_plotly['Ano'] = df_plotly['data'].dt.year
df_plotly['Mês'] = df_plotly['data'].dt.month
anos_disponiveis = sorted(df_plotly['Ano'].unique())
meses_disponiveis = sorted(df_plotly['Mês'].unique())

year_widget = widgets.Dropdown(options=anos_disponiveis, value=anos_disponiveis[0], description='Ano:')
month_widget = widgets.SelectMultiple(options=meses_disponiveis, value=[1, 2, 3], description='Mês(es):')
button = widgets.Button(description='Atualizar Gráfico', button_style='primary', icon='check')

# 2. NOVO: Criar um 'espaço reservado' (Output widget) para o nosso gráfico
# É aqui que o novo gráfico será desenhado a cada clique.
output_container = widgets.Output()

# 3. Definir a função que será chamada pelo botão
def on_button_click_recreate(b):
    # Pega os valores dos widgets de seleção
    ano = year_widget.value
    meses = month_widget.value
    
    # Filtra os dados
    df_filtrado = df_plotly[(df_plotly['Ano'] == ano) & (df_plotly['Mês'].isin(meses))]
    
    # ...dentro da sua função on_button_click_recreate...

    # Cria um GRÁFICO COMPLETAMENTE NOVO com os dados filtrados
    novo_grafico = px.line(
        df_filtrado,
        x='data',
        y='MediaPoluentes',
        color='cidade',
        markers=True,
        title=f"Poluição Diária em {ano}"
    )
    
    # Aplica o layout e o formato da data ao novo gráfico
    novo_grafico.update_layout(
        yaxis_title="Índice de Média de Poluentes"
    )
    novo_grafico.update_xaxes(
        title_text="Data",
        # --- MUDANÇA AQUI ---
        tickformat="%b %Y",  # Formato: Jan 2022, Fev 2022, etc.
        rangeslider_visible=False
    )
    
    # Limpa o conteúdo antigo e desenha o novo gráfico
    with output_container:
        clear_output(wait=True)
        display(novo_grafico)

# ...o resto do seu código continua igual...

# 4. Ligar o botão à nova função de recriação
button.on_click(on_button_click_recreate)

# 5. Exibir os controles e o 'espaço reservado' do gráfico
controles = HBox([year_widget, month_widget])
display(VBox([controles, button, output_container]))

# 6. Chamar a função uma vez manualmente para desenhar o gráfico inicial
on_button_click_recreate(None)

In [15]:
# --- GRÁFICO 2: COM LEGENDA DE QUALIDADE DO AR ---

legenda_texto = "Qualidade do Ar (Média de Poluentes) -> Boa: <25 | Moderada: 25-40 | Ruim: 40-55 | Muito Ruim: >55"

df_agregado = df_plotly.groupby('QualidadeAr')['casos_problemas_respiratorios'].mean().reset_index()

fig2_barras = px.bar(
    df_agregado,
    x='QualidadeAr',
    y='casos_problemas_respiratorios',
    color='QualidadeAr',
    text_auto='.0f',
    category_orders={'QualidadeAr': ['Boa', 'Moderada', 'Ruim', 'Muito Ruim']}, 
    
    # Adicionamos a legenda ao título usando a tag <br> para quebra de linha
    title=f"<b>Média de Casos Respiratórios por Nível de Qualidade do Ar</b><br><sup>{legenda_texto}</sup>", # <--- MUDANÇA AQUI
    
    labels={
        'QualidadeAr': 'Qualidade do Ar Classificada', 
        'casos_problemas_respiratorios': 'Média de Casos Respiratórios Diários'
    },
    template='plotly_white'
)

fig2_barras.show()

In [8]:
legenda_texto = "Qualidade do Ar (Média de Poluentes) -> Boa: <25 | Moderada: 25-40 | Ruim: 40-55 | Muito Ruim: >55"

fig3 = px.bar(
    df_plotly, 
    x='QualidadeAr', 
    color='QualidadeAr', 
    facet_col='cidade',
    category_orders={'QualidadeAr': ['Boa', 'Moderada', 'Ruim', 'Muito Ruim']},
    
    title=f'<b>Distribuição da Qualidade do Ar por Cidade</b><br><sup>{legenda_texto}</sup>',
    
    template='plotly_white',
    labels={'QualidadeAr': 'Qualidade do Ar'}
)
fig3.update_yaxes(title_text='Número de Dias')

fig3.show()